In [3]:
import pandas as pd
import numpy as np
import requests
import geopandas as gpd

In [4]:
pothole = pd.read_csv("https://data.cityofchicago.org/resource/7as2-ds3y.csv?$limit=1000000")

In [ ]:
print (pothole)

                  creation_date      status          completion_date  \
0       2018-12-18T00:00:00.000        Open                      NaN   
1       2018-12-18T00:00:00.000        Open                      NaN   
2       2018-12-18T00:00:00.000        Open                      NaN   
3       2018-12-18T00:00:00.000  Open - Dup                      NaN   
4       2018-12-18T00:00:00.000        Open                      NaN   
...                         ...         ...                      ...   
560473  2010-03-11T00:00:00.000   Completed  2011-03-28T00:00:00.000   
560474  2010-02-26T00:00:00.000   Completed  2011-10-04T00:00:00.000   
560475  2009-08-27T00:00:00.000   Completed  2011-01-25T00:00:00.000   
560476  2009-06-17T00:00:00.000   Completed  2011-06-07T00:00:00.000   
560477  2009-01-12T00:00:00.000   Completed  2011-10-06T00:00:00.000   

       service_request_number type_of_service_request current_activity  \
0                 18-03387505       Pothole in Street        

In [ ]:
# What season gets the most potholes filled?

In [5]:
patched = pd.read_csv("https://data.cityofchicago.org/resource/wqdh-9gek.csv?$limit=1000000")

In [ ]:
patched['request_date'] = pd.to_datetime(patched['request_date'])

In [ ]:
def month_to_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Autumn'

In [ ]:
patched['season'] = patched['request_date'].dt.month.apply(month_to_season)

In [ ]:
season_potholes_filled = patched.groupby('season')['number_of_potholes_filled_on_block'].sum()

In [ ]:
print(patched.head())

                 address        request_date          completion_date  \
0     3948 N WOLCOTT AVE 2024-04-19 09:09:58  2024-04-19T14:24:37.000   
1     4107 N ASHLAND AVE 2024-04-19 09:29:50  2024-04-19T14:23:10.000   
2      1556 W LELAND AVE 2024-04-19 09:49:08  2024-04-19T14:22:19.000   
3   6719 N HERMITAGE AVE 2024-04-19 11:42:09  2024-04-19T14:21:27.000   
4  6800 N RAVENSWOOD AVE 2024-04-19 10:59:35  2024-04-19T14:20:09.000   

   number_of_potholes_filled_on_block   latitude  longitude  \
0                                  20  41.953822 -87.676384   
1                                   3  41.956398 -87.668590   
2                                   2  41.967140 -87.668609   
3                                  32  42.004134 -87.673563   
4                                  32  42.005360 -87.675759   

                                   location  season  
0  POINT (-87.676383878216 41.953822261036)  Spring  
1  POINT (-87.668590500002 41.956398000941)  Spring  
2   POINT (-87.66860

In [ ]:
print(season_potholes_filled)

season
Autumn    208122
Spring    993908
Summer    313235
Winter    625338
Name: number_of_potholes_filled_on_block, dtype: int64


In [ ]:
average_duration_by_season = patched.groupby('season')['number_of_potholes_filled_on_block'].mean()
print(average_duration_by_season.max())

13.419583063296608


In [ ]:
# Which wards have the most pothole requests?

In [ ]:
pothole.groupby('ward')['type_of_service_request'].count().sort_values(ascending = False)

ward
41.0    21637
45.0    16788
19.0    16534
23.0    16398
32.0    16173
13.0    15765
39.0    15699
8.0     15336
42.0    14730
34.0    14528
21.0    14126
2.0     13790
18.0    13761
36.0    13687
27.0    13221
38.0    12878
50.0    12760
14.0    12244
10.0    12146
40.0    11965
29.0    11730
6.0     11460
35.0    10732
28.0    10693
25.0    10621
1.0     10545
5.0     10438
9.0     10380
11.0     9743
12.0     9620
49.0     9551
47.0     9458
31.0     9351
17.0     9055
24.0     9044
37.0     8942
30.0     8436
26.0     8358
43.0     8231
7.0      7997
33.0     7922
16.0     7825
3.0      7611
20.0     7541
15.0     7334
48.0     7076
4.0      7042
22.0     6966
44.0     6373
46.0     4340
0.0      1564
Name: type_of_service_request, dtype: int64

In [ ]:
# How many potholes are filled in each ward? (with patched data)

In [ ]:
patched.groupby('number_of_potholes_filled_on_block').size().sort_values(ascending = False)

number_of_potholes_filled_on_block
1      20589
2      18099
3      14280
5      12556
4      12028
       ...  
212        1
214        1
215        1
218        1
467        1
Length: 230, dtype: int64

In [ ]:
# According to the patched data, how long does it take for a pothole to get patched?

In [ ]:
patched['request_date'] = pd.to_datetime(patched['request_date'])
patched['completion_date'] = pd.to_datetime(patched['completion_date'])

In [ ]:
patched['duration'] = (patched['completion_date'] - patched['request_date']).dt.days

In [ ]:
print(patched[['request_date', 'completion_date', 'duration']].head())

         request_date     completion_date  duration
0 2024-04-19 09:09:58 2024-04-19 14:24:37         0
1 2024-04-19 09:29:50 2024-04-19 14:23:10         0
2 2024-04-19 09:49:08 2024-04-19 14:22:19         0
3 2024-04-19 11:42:09 2024-04-19 14:21:27         0
4 2024-04-19 10:59:35 2024-04-19 14:20:09         0


In [ ]:
average_duration = patched['duration'].mean()
print(f"The average duration it takes to patch a pothole is {average_duration} days.")

The average duration it takes to patch a pothole is 20.197752871331076 days.


In [ ]:
median_duration = patched['duration'].median()
print(f"The median duration it takes to patch a pothole is {median_duration} days.")

The median duration it takes to patch a pothole is 4.0 days.


In [ ]:
print(patched['duration'].describe())

count    180230.000000
mean         20.197753
std          40.150847
min           0.000000
25%           0.000000
50%           4.000000
75%          18.000000
max         553.000000
Name: duration, dtype: float64


In [ ]:
# Show the melted date (PATCHED)

In [ ]:
id_vars = ['address', 'location']
value_vars = ['request_date', 'number_of_potholes_filled_on_block']

In [ ]:
melted_data = patched.melt(id_vars=id_vars, value_vars=value_vars, var_name='variable', value_name='value')

In [ ]:
print(melted_data)

                      address                                  location  \
0          3948 N WOLCOTT AVE  POINT (-87.676383878216 41.953822261036)   
1          4107 N ASHLAND AVE  POINT (-87.668590500002 41.956398000941)   
2           1556 W LELAND AVE   POINT (-87.66860942898 41.967140060035)   
3        6719 N HERMITAGE AVE  POINT (-87.673563000002 42.004134000941)   
4       6800 N RAVENSWOOD AVE  POINT (-87.675758982353 42.005360096258)   
...                       ...                                       ...   
360455         3911 W 61ST ST              POINT (-87.720834 41.782065)   
360456         3908 W 61ST ST              POINT (-87.720808 41.782588)   
360457         3900 W 62ND PL              POINT (-87.720513 41.779863)   
360458         3847 W 61ST PL              POINT (-87.719803 41.781169)   
360459    6400 S MAJOR Avenue  POINT (-87.764136509305 41.776112376302)   

                                  variable                value  
0                             req

In [ ]:
# LOOP (Patched)

In [ ]:
total_potholes_per_address = {}

In [ ]:
for index, row in patched.iterrows():
    address = row['address']
    potholes_filled = row['number_of_potholes_filled_on_block']
if address not in total_potholes_per_address:
        total_potholes_per_address[address] = potholes_filled
else:total_potholes_per_address[address] += potholes_filled

In [ ]:
for address, total_potholes in total_potholes_per_address.items():
    print(f"Address: {address}, Total Potholes Filled: {total_potholes}")

In [ ]:
# Demonstrate an F String (Patched)

In [ ]:
for index, row in patched.iterrows():
  if index < 100:
    summary_string = f"Address: {row['address']}, Request Date: {row['request_date']}, Potholes Filled: {row['number_of_potholes_filled_on_block']}, Location: {row['location']}"
    print(summary_string)
  else:
    break

How many potholes have been patched, how many requests are still open, random sample maker of open cases, how many requests had no problem found

In [ ]:
sample_pothole = pothole.sample(5)

In [ ]:
for column in sample_pothole.iterrows():
  if column[1]['status'] == 'Completed':
    print('its completed!')
  elif column[1]['status'] == 'Completed - Dup':
    print('its completed!')
  else:
    print('it hasnt been completed')

In [ ]:
open_requests = pothole['status'].isin(['Open', 'Open - Dup'])

In [ ]:
open_requests.value_counts()

In [ ]:
potholes_patched = pothole['most_recent_action'] == 'Pothole Patched'

In [ ]:
potholes_patched.value_counts()

In [ ]:
potholes_no_problem = pothole['most_recent_action'] == 'No Problem Found'

In [ ]:
potholes_no_problem.value_counts()

In [ ]:
# How many duplicate requests are in the pothole request data?

In [ ]:
duplicates = pothole.duplicated()

In [ ]:
num_duplicates = duplicates.sum()

In [ ]:
if num_duplicates > 0:
  print(f'there are {num_duplicates} duplicate rows in the dataset.')
else:
  print('there are no duplicate rows in the dataet.')

In [ ]:
# New column showing the duplicates

In [ ]:
duplicates_col = pothole.duplicated(subset = 'service_request_number', keep=False)

In [ ]:
pothole['is_duplicated'] = duplicates_col

In [ ]:
pothole.head()

In [ ]:
# Amount of pothole requests per address?

In [ ]:
pothole['street_address'].value_counts().reset_index()

In [ ]:
#
#Are there certain neighborhoods or points in time that occur more often?

In [ ]:
pothole.columns

In [ ]:
patched.columns

In [ ]:
print(patched.completion_date)

In [ ]:
pothole['creation_date'] = pd.to_datetime(pothole['creation_date'])

In [ ]:
pothole['completion_date'] = pd.to_datetime(pothole['completion_date'])

In [ ]:
pothole['ph_month'] = pothole['creation_date'].dt.month
pothole['ph_weekday'] = pothole['creation_date'].dt.weekday
pothole['ph_day'] = pothole['creation_date'].dt.day
pothole['ph_year'] = pothole['creation_date'].dt.year

In [ ]:
pothole[pothole['completion_date'] < '2019-12-31'].head(2)

In [ ]:
pothole['count'] = 1

In [ ]:
pothole.groupby(['ph_year', 'ph_month'])['count'].sum()

In [ ]:
pothole.groupby(['ph_year', 'ph_month'])['count'].sum().reset_index()

In [ ]:
pothole.columns

In [ ]:
frequencymaybe = pothole.groupby(['ph_year', 'ph_month', 'ph_day'])['count'].sum()

In [ ]:
freq = (frequencymaybe.sort_values(ascending=False))

In [ ]:
print(freq)

In [ ]:
# - above answers -
#THE MONTH WITH THE MOST REPORTS IS MARCH - only 3 years from 2011-2018 had March not being the most reported
  #FURTHERMORE, THE MONTHS WITH THE HIGHEST REPORTS PER DAY ARE IN THE WINTER
  # all according to dataset POTHOLE

In [ ]:
#
# How long does it take for a pothole to get patched?
  #31.4 days - According to dataset POTHOLE
    # median number of days is 8 - according to dataset POTHOLE

In [ ]:
pothole['request_complete'] = pothole['completion_date'] - pothole['creation_date']

In [ ]:
pothole['request_complete'] = pothole['request_complete'].dt.days

In [ ]:
pothole.columns

In [ ]:
print(pothole.request_complete)

In [ ]:
lencompreq = pothole['request_complete'].mean()

In [ ]:
print(lencompreq)

In [ ]:
medcomreq = pothole['request_complete'].median()

In [ ]:
print(medcomreq)

In [1]:
#how frequently pothole requests are made, median and groupbys

In [7]:
pothole_data['creation_date'] = pd.to_datetime(pothole_data['creation_date'])

NameError: name 'pothole_data' is not defined

In [ ]:
median_requests = pothole_data['creation_date'].median()

In [ ]:
print(median_requests)

In [ ]:
grouped_data = pothole_data.groupby('location')['number_of_potholes_filled_on_block'].mean()

In [ ]:
requests_per_location = pothole_data['location'].value_counts()

In [ ]:
requests_per_location = pothole_data.groupby('location').size()

In [ ]:
avg_potholes_per_request = pothole_data.groupby('location')['number_of_potholes_filled_on_block'].mean()

In [ ]:
print(median_requests)

In [ ]:
med_potholes_patched = pothole_data['number_of_potholes_filled_on_block'].median()

In [ ]:
print("Median number of potholes patched:", med_potholes_patched)